In [101]:
# Imports
from folium import plugins

import folium
import ipywidgets
import geocoder
import geopy
import numpy as np
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster # for clustering the markers

In [102]:
# School Data
df = pd.read_excel('ava.xlsx', sheet_name='Sheet3')


In [105]:
import re

# Use colors supported by folium
color_dict = {
    "yellow": "lightred",
    "green": "lightgreen",
    "blue": "blue",
    "pink": "pink",
    "grey": "gray",
    "maroon": "cadetblue",
    "black": "black",
    "red": "red",
    "purple": "purple",
    "gold": "beige",
    "orange": "orange",
}


# Show schools with the same course in the same color
map1 = folium.Map(location=[-1.28333, 36.81667], zoom_start=7, width=1000, height=1000, control_scale=True)

# add a marker for every record in the filtered data, use a clustered view
marker_cluster = MarkerCluster().add_to(map1) # create marker clusters

for index, row in df.iterrows():
    name, lat, long, group_name, color = row[1], row[2], row[3], row[4], row[5]
    folium.Marker([lat, long],
                  icon=folium.Icon(color=color_dict[color], icon='home'),
                  popup=group_name, tooltip=name).add_to(marker_cluster)

# Display the map
map1.save('Kenya_All_Schools_Final.html')
map1


In [128]:
# Group by discipline
grouped_disciplines = df.groupby('Discipline')

kenya = folium.Map(location=[-1.28333, 36.81667], zoom_start=7, control_scale=True)

for name, group in grouped_disciplines:  
    feature_group=folium.FeatureGroup(name=name, show=True)
    kenya.add_child(feature_group)
    latitudes = group['Latitude'].astype(float).tolist()
    longitudes = group['Longitude'].astype(float).tolist()
    school_names = group['School'].tolist()
    colors = group['Color'].tolist()
    for lat, long, school_name, color in zip(latitudes, longitudes, school_names, colors):
        folium.Marker([lat, long], 
                      popup=name,
                      icon=folium.Icon(color=color_dict[color], icon='home'),
                      tooltip=school_name).add_to(feature_group)
   
folium.LayerControl().add_to(kenya) 

# Display the map with the overlay
kenya.save('index.html')
kenya

In [127]:
## Legends

#-*- coding: utf-8 -*-

legend_html =   '''
                <div style="position: fixed; 
                            bottom: 50px; left: 50px; width: 100px; height: 90px; 
                            border:2px solid grey; z-index:9999; font-size:14px;
                            ">&nbsp; Cool Legend <br>
                              &nbsp; East &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i><br>
                              &nbsp; West &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i>
                </div>
                ''' 

kenya.get_root().html.add_child(folium.Element(legend_html))
kenya